# **DATA PREPROCESSING BASED ON EDA INSIGHTS**

This notebook implements preprocessing steps based on the comprehensive EDA findings and recommendations. We'll follow the evidence-based approach from the EDA report to ensure our preprocessing aligns with the data patterns discovered.
Based on the EDA report, we will:

1. **Handle Skewed Variables** - Log-transform `ApplicantIncome` and `CoapplicantIncome`
3. **Feature Engineering** - Create Total_income, Loan_to_income, and interaction features.
4. **Encoding categorical columns**
5. **Scaling** - RobustScaler 
6. **Target Handling** - Classification approach with stratified splits
4. **Feature Selection** - Keep high-signal features, evaluate low-signal ones
7. **Splitting into target and Features**


**Key EDA Evidence to Implement**

- **High-signal features**: `Credit_History, ApplicantIncome, CoapplicantIncome, LoanAmount, Property_Area`, and derived variables such as Total_Income and Loan_to_Income show strong or meaningful relationships with loan approval outcomes.
- **Low-signal features**: `Gender, Married, Dependents, Self_Employed, and Loan_Amount_Term `display weak or negligible influence on loan approval and may be deprioritized during preprocessing.
- **Skewed variables**: `ApplicantIncome` and `CoapplicantIncome` (log-transform)
- **Feature engineering**: Total_Income
Loan_to_Income, Loan_Term_Years, Has_Coapplicant, Dependents_Num.



#### **1. Import Libraries and load the data**

In [132]:
# Core libraries
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import warnings
warnings.filterwarnings('ignore')

# Preprocessing libraries
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix


# Statistical libraries
from scipy import stats
from scipy.stats import zscore, skew

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

print('All libraries imported successfully')


All libraries imported successfully


In [133]:
# Load the dataset
loan_train = pd.read_csv('cleaned_home_data.csv')

loan_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,1
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1


In [134]:
# Create a copy of the dataset for preprocessing
df_processed = loan_train.copy()

In [135]:
# Set the Loan_ID as the index 
df_processed.set_index('Loan_ID', inplace=True)

In [136]:
# # Map the Loan_Status from object to int to check correlation 
# df_processed['Loan_Status'] = df_processed['Loan_Status'].map({'Y': 1, 'N': 0})


In [137]:
# # Display output
# df_processed['Loan_Status']

#### **2. EDA-Based Data Assessment**

In [138]:
# 1. Check for missing values (EDA showed no missing values)
print("\n1. Missing Values:")
missing_values = df_processed.isna().sum()
if missing_values.sum() > 0:
    print(missing_values[missing_values > 0])
else:
    print("No missing values found as expceted from the EDA")


# 2. Check for duplicates
print("\n2. Duplicate Rows:")
duplicates = df_processed.duplicated().sum()
print(f"Number of duplicated rows: {duplicates}")
if duplicates.sum() > 0:
    print(f"Percentage of duplicates: {(duplicates/len(df_processed))* 100:.2f}")


# 3. Check for skewness for variables identified in EDA as right-skewed
print(f"\n3. Skewness Analysis (EDA identified right-skewed variables):")
skewed_var = ['ApplicantIncome','CoapplicantIncome','LoanAmount']
for var in skewed_var:
    if var in df_processed.columns:
        skewness = skew(df_processed[var])
        print(f"{var}: Skewness = {skewness:.3f} ({'right_skewed' if skewness > 0.5 else 'Approx normal'})")
    

# 4. Check the correlation with target (EDA evidence)
print("\n 4. Correlation with quality (EDA Evidence):")
correlations = df_processed.select_dtypes(include=['int64', 'float64']).corr()['Loan_Status'].sort_values(key=abs, ascending=False)
print("High signal features (|Correlation| > 0.2)")
high_signal = correlations[abs(correlations) > 0.2].drop('Loan_Status')
for feature, corr in high_signal.items():
    print(f"{feature}: {corr:.3f}")

print("\n Low-signal features (|correlated| < 0.1)")
low_signal = correlations[abs(correlations) < 0.1]
for feature, corr in low_signal.items():
    print(f"{feature}: {corr:.3f}")



1. Missing Values:
No missing values found as expceted from the EDA

2. Duplicate Rows:
Number of duplicated rows: 0

3. Skewness Analysis (EDA identified right-skewed variables):
ApplicantIncome: Skewness = 6.524 (right_skewed)
CoapplicantIncome: Skewness = 7.473 (right_skewed)
LoanAmount: Skewness = 2.736 (right_skewed)

 4. Correlation with quality (EDA Evidence):
High signal features (|Correlation| > 0.2)
Credit_History: 0.541

 Low-signal features (|correlated| < 0.1)
CoapplicantIncome: -0.059
LoanAmount: -0.033
Loan_Amount_Term: -0.023
Dependents: 0.010
ApplicantIncome: -0.005


#### **3. Handle Duplicated values**

In [139]:
# Remove duplicates if any (EDA didn't report duplivcates, but let's be thorough)
if duplicates > 0:
    print(f"Removing {duplicates} duplicate rows...")
    df_processed = df_processed.drop_duplicates()
    print(f"Dataset shape after removing duplicates: {df_processed.shape}")
else:
    print("✓ No duplicates to remove (as expected from EDA)")

✓ No duplicates to remove (as expected from EDA)


#### **3. Feature Engineering (EDA Recommendation)**

Feature engineering recommedndation from the EDA:

- Total_Income = ApplicantIncome + Coapplicantincome
- Loan_Amount_Term_Year = Loan_Amount_Term / 12
- EMI = Loan_amount / Loan_Amount_Term_year
- Has_Coapplicant = Yes or No
- Dependents_Num
- Income category




**Total Income**

In [140]:
# Feature Engineering
print("FEATURE ENGINEERING BASED ON EDA RECOMMENDATIONS")


# Total_Income = ApplicantIncome + CoapplicantIncome
if all(col in df_processed for col in ["ApplicantIncome", "CoapplicantIncome"]):
    df_processed["Total_income"] = df_processed["ApplicantIncome"] + df_processed["CoapplicantIncome"]
    print("Total Income column created!")
else:
    print("Could not create 'Total_income': missing one or ore income columns")


print(f"Dataset shape after feature engineering: {df_processed.shape}")
print("Feature engineering complete. Ready for encoding and scaling. \n")



FEATURE ENGINEERING BASED ON EDA RECOMMENDATIONS
Total Income column created!
Dataset shape after feature engineering: (614, 13)
Feature engineering complete. Ready for encoding and scaling. 



In [141]:
df_processed.head(2)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_income
Loan_ID,,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,1,5849.0
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,6091.0


**Loan Amount Term Year**

In [142]:
# # Converting the Loan Amount type back toInt 64
df_processed["Loan_Amount_Term"] = df_processed["Loan_Amount_Term"].astype(int)
df_processed["Loan_Amount_Term"] 

Loan_ID
LP001002    360
LP001003    360
LP001005    360
LP001006    360
LP001008    360
           ... 
LP002978    360
LP002979    180
LP002983    360
LP002984    360
LP002990    360
Name: Loan_Amount_Term, Length: 614, dtype: int64

In [143]:
# Calculating the years of the loan amount term from the months
df_processed["Loan_Amount_Term_year"] = round((df_processed["Loan_Amount_Term"] / 12) , 1).astype(int)
df_processed["Loan_Amount_Term_year"].head()

Loan_ID
LP001002    30
LP001003    30
LP001005    30
LP001006    30
LP001008    30
Name: Loan_Amount_Term_year, dtype: int64

In [144]:
df_processed.head(2)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_income,Loan_Amount_Term_year
Loan_ID,,,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360,1.0,Urban,1,5849.0,30
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360,1.0,Rural,0,6091.0,30


**Equated Monthly Instalment = Loan_amount / Loan_Amount_Term_year**

In [145]:
# Equated Monthly Instalment = Loan_amount / Loan_Amount_Term_year
if all(col in df_processed for col in ["LoanAmount", "Loan_Amount_Term_year"]):
    df_processed["EMI"] = df_processed["LoanAmount"] + df_processed["Loan_Amount_Term_year"]
    print("Equated Monthly Instalment(EMI) column created!")
else:
    print("Could not create 'EMI': missing one or ore income columns")

Equated Monthly Instalment(EMI) column created!


In [146]:
df_processed.head(2)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_income,Loan_Amount_Term_year,EMI
Loan_ID,,,,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360,1.0,Urban,1,5849.0,30,158.0
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360,1.0,Rural,0,6091.0,30,158.0


**Has_Coapplicant - Married or Not**

In [ ]:
df_processed.Married.unique()

array(['No', 'Yes'], dtype=object)

| Categorical | Numeric   | Meaning     |
| ----------- | ----------| ------------|
| Yes         | 1         | Married     |
| No          | 0         | Not Married |


In [148]:
def map_married_category(x):
    if x == 'Yes':
        return '1'
    else:
        return '0'

# Lets apply the function
df_processed['Has_Coapplicant'] = df_processed['Married'].apply(map_married_category)

In [149]:
df_processed.head(2)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_income,Loan_Amount_Term_year,EMI,Has_Coapplicant
Loan_ID,,,,,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360,1.0,Urban,1,5849.0,30,158.0,0
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360,1.0,Rural,0,6091.0,30,158.0,1


**Income category**

In [154]:
def map_income_category(income):
    if income <= 1500:
        return "Low"
    elif (income > 1500) and (income <= 5000):
        return "Medium"
    elif income > 5000:
        return "High"
    else:
        return "Very High"

# Lets apply the function
df_processed['income_category'] = df_processed['ApplicantIncome'].apply(map_income_category)

In [155]:
df_processed.head(2)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_income,Loan_Amount_Term_year,EMI,Has_Coapplicant,ApplicantIncome_log,CoapplicantIncome_log,LoanAmount_log,income_category
Loan_ID,,,,,,,,,,,,,,,,,,,,
LP001002,Male,No,0.0,Graduate,No,5849.0,0.0,128.0,360.0,1.0,Urban,1,5849.0,30.0,158.0,0,8.674026,0.000000,4.85203,High
LP001003,Male,Yes,1.0,Graduate,No,4583.0,1508.0,128.0,360.0,1.0,Rural,0,6091.0,30.0,158.0,1,8.430109,7.319202,4.85203,Medium


#### **4. Log-Transform Skewed Variables (EDA Recommendation)**

Based on EDA findings, transform the right-skewed variables identified

In [150]:
# # Log-transfrorm skewed varibaled as recommended by EDA
def handle_skewed_var(df):
    # Variables to log-transform based on EDA findings
    skewed_var = ['ApplicantIncome','CoapplicantIncome','LoanAmount']
    for var in skewed_var:
            # Checks if variable has zero or negative values
        min_val = df_processed[var].min()
        if min_val <=0:
            # Use log1p for variables with zeros
            df_processed[f'{var}_log'] = np.log1p(df_processed[var])
            print(f'✓ {var}: Applied log1p transformation (had {min_val:.3f} minimum value)')

        else:
            # Use log for positive values only
            df_processed[f'{var}_log'] = np.log(df_processed[var])
            print(f"✓ {var}: Applied log transformation")

        # Check for skewness before and after
        original_skew = skew(df_processed[var])
        transformed_skew = skew(df_processed[f'{var}_log'])
        print(f" Original Skewness: {original_skew:.3f} -> Transformed skewness: {transformed_skew:.3f}")
    return df



In [151]:
handle_skewed_var(df_processed)

✓ ApplicantIncome: Applied log transformation
 Original Skewness: 6.524 -> Transformed skewness: 0.478
✓ CoapplicantIncome: Applied log1p transformation (had 0.000 minimum value)
 Original Skewness: 7.473 -> Transformed skewness: -0.173
✓ LoanAmount: Applied log transformation
 Original Skewness: 2.736 -> Transformed skewness: -0.195


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_income,Loan_Amount_Term_year,EMI,Has_Coapplicant,ApplicantIncome_log,CoapplicantIncome_log,LoanAmount_log
Loan_ID,,,,,,,,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360,1.0,Urban,1,5849.0,30,158.0,0,8.674026,0.000000,4.852030
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360,1.0,Rural,0,6091.0,30,158.0,1,8.430109,7.319202,4.852030
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360,1.0,Urban,1,3000.0,30,96.0,1,8.006368,0.000000,4.189655
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360,1.0,Urban,1,4941.0,30,150.0,1,7.856707,7.765993,4.787492
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360,1.0,Urban,1,6000.0,30,171.0,0,8.699515,0.000000,4.948760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360,1.0,Rural,1,2900.0,30,101.0,0,7.972466,0.000000,4.262680
LP002979,Male,Yes,3,Graduate,No,4106,0.0,40.0,180,1.0,Rural,1,4106.0,15,55.0,1,8.320205,0.000000,3.688879
LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360,1.0,Urban,1,8312.0,30,283.0,1,8.996157,5.484797,5.533389


In [152]:
# Outlier treatment based on EDA recommendations
print("----Outlier Treatment (IQR-clipping method)----")
print("EDA recommend IQR-clipping for extreme values to preserve data points")

# Define numerical columns (excluding target)
num_col = df_processed.select_dtypes(include=[np.number]).columns.tolist()
if 'Loan_Status' in num_col:
    num_col.remove('Loan_Status')

print(f"Treating ouliers in {len(num_col)} numerical features...")

# Apply IQR_clipping method
outliers_clipped = 0
for col in num_col:
    Q1 = df_processed[col].quantile(0.25)
    Q3 = df_processed[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Countoutliers before clipping
    outliers_before = ((df_processed[col] < lower_bound) | (df_processed[col] > upper_bound)).sum()

    if outliers_before > 0:
        # Clip outliers
        df_processed[col] = np.where(df_processed[col] < lower_bound, lower_bound, df_processed[col])
        df_processed[col] = np.where(df_processed[col] > upper_bound, upper_bound, df_processed[col])
        


----Outlier Treatment (IQR-clipping method)----
EDA recommend IQR-clipping for extreme values to preserve data points
Treating ouliers in 12 numerical features...
